## Communicating with own knowledge base using `OpenAI`

In [2]:
import credentials
import time
import re
import os
os.environ["OPENAI_API_KEY"] = credentials.openai_api

import openai
import tiktoken

from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.node_parser import SimpleNodeParser
from llama_index import StorageContext, load_index_from_storage

# for low-level API calls
from llama_index import VectorStoreIndex, ResponseSynthesizer
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor

# langchain helpers

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader

### VectorDB from documents

In [4]:
path = '../docs/docs_to_index/'

loader = PyPDFDirectoryLoader(path)
docs = loader.load()

In [6]:
len(docs)

206

In [ ]:
# trick to help with context
# add to each page the document name

In [38]:
for i in docs:
    i.page_content = i.metadata['source'].split("\\")[-1] + ' --- ' + i.page_content

Apply OpenAI Embedder

In [40]:
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(docs, embeddings)

In [ ]:
query = 'Are dogs allowed on RyanAir?'
query_embedded = embeddings.embed_query(query)

#sim_docs = db.similarity_search_with_score(query, ) #score the lower the better here (L2)
#sim_docs = db.similarity_search_with_score_by_vector(query_embedded, k = 3)
sim_docs = db.max_marginal_relevance_search_by_vector(query_embedded, k = 3)

In [ ]:
sim_docs

[Document(page_content='ryanair_conditions.pdf --- 6/11/23, 12:22 AM General Terms & Conditions of Carriage\nhttps://www .ryanair .com/gb/en/useful-info/help-centre/terms-and-conditions 49/118Throughout the journe y, the dog must wear a jack et or harness that identi\x00es it as a guide dog or\nassistance dog.\nThe dog must sit on the \x00oor at y our f eet. \xa0\nYou must tell us that y ou will be tr avelling with a guide dog or assistance dog befor ehand, and\npreferably when y ou mak e your booking. After y ou ha ve made y our booking, y ou can tell us about\nthe dog thr ough the ‘My bookings’ section on our website or b y phoning our\xa0 Special Assistance\nLine .\nThe dog must wear a suitable harness (pr ovided b y you) attached t o your seatbelt during tak e-off,\nlanding and turbulence. Ther e ma y be other suitable equipment with alternativ e attachment\npoints.\nEmotional suppor t or ther apy dogs that ar e not r ecognised as assistance dogs b y the abo ve\norganisations ar e 

Save FAISS DB

In [82]:
db.save_local("../docs/docs_to_index_vectorDB/")

### Construct prompt with sources

In [77]:
# Davinci uses a particular set of tokens to indicate turns in conversations
prompt_prefix = """<|im_start|>system
Assistant helps people answer questions about a wide variety of documents. 
Answer ONLY with the facts listed in the list of sources below. If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question. 
Each source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. Use square brackets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].

Sources:
{sources}

<|im_end|>"""

turn_prefix = """
<|im_start|>user
"""

turn_suffix = """
<|im_end|>
<|im_start|>assistant
"""

prompt_history = turn_prefix

history = []

summary_prompt_template = """Below is a summary of the conversation so far, and a new question asked by the user that needs to be answered by searching in a knowledge base. 
Generate a search query based on the conversation and the new question. 
Do not include source names, file names in the search query.
Pay more attention to the new question, rather than the summary.

Summary:
{summary}

Question:
{question}

Search query:
"""

In [80]:
# Execute this cell multiple times updating user_input to accumulate chat history
user_input = "What is RyanAir's cancellation policy?"
user_input = "What is WizzAir's cancellation policy?"
user_input = "What is EasyJet's cancellation policy?"

#embed user input for relevancy search
query_embedded = embeddings.embed_query(user_input)


# Exclude category, to simulate scenarios where there's a set of docs you can't see
#exclude_category = None

#if len(history) > 0:
#    completion = openai.Completion.create(
#        engine='text-davinci-003',
#        prompt=summary_prompt_template.format(summary="\n".join(history), question=user_input),
#        temperature=0.7,
#        max_tokens=32,
#        stop=["\n"])
#    search = completion.choices[0].text
#else:
#    search = user_input

search = user_input

# Alternatively simply use search_client.search(q, top=3) if not using semantic search
print("Searching:", search)
print("-------------------")


sim_docs = db.max_marginal_relevance_search_by_vector(query_embedded, k = 3)

results = [doc.metadata['source'].split("\\")[-1] + "-page-" + str(doc.metadata['page'] )+ ": " + doc.page_content.replace("\n", "").replace("\r", "") for doc in sim_docs]
content = "\n".join(results)

prompt = prompt_prefix.format(sources=content) + prompt_history + user_input + turn_suffix

completion = openai.Completion.create(
    engine='text-davinci-003', 
    prompt=prompt, 
    temperature=0.0, 
    max_tokens=1024,
    stop=["<|im_end|>", "<|im_start|>"])

prompt_history += user_input + turn_suffix + completion.choices[0].text + "\n<|im_end|>" + turn_prefix
history.append("user: " + user_input)
history.append("assistant: " + completion.choices[0].text)

print("\n-------------------\n".join(history))
print("\n-------------------\nPrompt:\n" + prompt)

Searching: What is EasyJet's cancellation policy?
-------------------
user: What is RyanAir's cancellation policy?
-------------------
assistant: According to [ryanair_conditions.pdf-page-16], if your flight is cancelled or delayed by two hours or more, RyanAir will give you information on your rights, including your rights to compensation and assistance. If we do not let you board the plane for a flight you have booked (as set out in the booking confirmation or itinerary) because a space is no longer available, we will compensate you in line with any relevant law that applies. We will give you information about your rights in these circumstances, particularly your rights relating to practical help and compensation. If an immediate family member who is not travelling with you dies within 28 days of your booked flight, you may claim a refund equal to the fare you paid for the particular flight (or flights) you don't take as a result, plus any associated taxes, fees and charges that you 

In [54]:
def direct_OpenAI_QA(context, question, model = 'gpt-3.5-turbo'):

    prompt = f"""Use the following pieces of context to answer the users question. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer

    {context}

    Question: 
    {question}

    Answer:"""

    message = [{"role": "user", "content": prompt}]

    result = openai.ChatCompletion.create(model=model,  
                                          messages = message, 
                                          temperature=0, 
                                          top_p=1)
    
    completion = result['choices'][0]['message']['content']
    usage_info = result['usage'].to_dict()

    return completion, usage_info

### LlamaIndex on documents

In [ ]:
# load docs and build index

documents = SimpleDirectoryReader('../docs/docs_to_index/').load_data()
index = VectorStoreIndex.from_documents(documents)

In [ ]:
# persist on computer
index.storage_context.persist(persist_dir="../docs/docs_to_index_vectorDB/")

# rebuild storage context
#storage_context = StorageContext.from_defaults(persist_dir="<persist_dir>")

# load index
#index = load_index_from_storage(storage_context)

In [ ]:
# check nodes

parser = SimpleNodeParser()
nodes = parser.get_nodes_from_documents(documents)

In [ ]:
# query index

query_engine = index.as_query_engine()
response = query_engine.query("Who are company representatives?")

In [ ]:
response.extra_info

{'e8e24678-1ea5-45e7-a108-2b822e220ad2': {'page_label': '7',
  'file_name': 'Law_Insider_eidos-therapeutics-inc_contract_Filed_23-03-2018_Contract.pdf'},
 'a7c216ee-a8ce-421f-8464-9d2d181b4045': {'page_label': '8',
  'file_name': 'Law_Insider_eidos-therapeutics-inc_contract_Filed_23-03-2018_Contract.pdf'}}

In [ ]:
print(response.get_formatted_sources())

> Source (Doc id: e8e24678-1ea5-45e7-a108-2b822e220ad2): page_label: 7
file_name: Law_Insider_eidos-therapeutics-inc_contract_Filed_23-03-2018_Contract.pd...

> Source (Doc id: a7c216ee-a8ce-421f-8464-9d2d181b4045): page_label: 8
file_name: Law_Insider_eidos-therapeutics-inc_contract_Filed_23-03-2018_Contract.pd...


In [ ]:
[i.score for i in response.source_nodes]

[0.7528456148084678, 0.7514728258301441]

In [ ]:
print(response.response)


Company representatives are Christine Siu, Chief Financial Officer of Eidos Therapeutics, Inc., and Neil Kumar, Chief Executive Officer of BridgeBio Pharma LLC.


Low level API

In [ ]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index, 
    similarity_top_k=3,
    retriever_mode = 'embedding' # default
)

# configure response synthesizer
response_synthesizer = ResponseSynthesizer.from_args(
    node_postprocessors=[
        SimilarityPostprocessor(similarity_cutoff=0.6)
    ]
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query("Who are company representatives?")
print(response)


Company representatives are Christine Siu (Chief Financial Officer) and Neil Kumar (Chief Executive Officer).


In [ ]:
[i.score for i in response.source_nodes]

[0.7528456148084678, 0.7514728258301441, 0.7402034007180792]

In [ ]:
retriever = VectorIndexRetriever(
    index=index, 
    similarity_top_k=3,
    retriever_mode = 'embedding' # default
)